# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file_location = "../WeatherPy/Output/weather_data.csv"

weather_df = pd.read_csv(file_location)
weather_df.drop(weather_df.columns[[0]], axis = 1, inplace = True)
weather_df.head()

,City,Country,Temp,Humidity,Lat,Long,Cloud,Wind,Date
0,Torbay,CA,63.00,41,47.67,-52.73,90,24.16,1592435924
1,Ambon City,ID,80.60,83,-3.70,128.18,75,4.70,1592435938
2,Puyang,CN,82.44,77,29.46,119.89,85,5.12,1592435939
3,Menomonie,US,84.20,34,44.88,-91.92,1,12.75,1592435897
4,Salalah,OM,84.20,89,17.02,54.09,40,2.24,1592435939


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

In [4]:
coords = weather_df[["Lat", "Long"]]
humidity = weather_df["Humidity"]

fig = gmaps.figure()

heatLayer = gmaps.heatmap_layer(coords,weights=humidity)
fig.add_layer(heatLayer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:

hotel_df = weather_df[(weather_df["Temp"] > 70) & (weather_df["Temp"] < 80) & 
                             (weather_df["Wind"] < 10) & (weather_df["Cloud"] == 0)]
hotel_df = hotel_df.dropna()
hotel_df

,City,Country,Temp,Humidity,Lat,Long,Cloud,Wind,Date
50,Corrente,BR,74.66,79,-3.74,-43.34,0,1.66,1592435953
102,Mossamedes,AO,71.44,60,-15.20,12.15,0,6.55,1592435849
137,Beyneu,KZ,77.49,19,45.32,55.20,0,9.66,1592435883
212,Kununurra,AU,71.60,45,-15.77,128.73,0,8.05,1592435997
230,Kiryat Gat,IL,75.99,45,31.61,34.76,0,3.71,1592435999
271,Sīrjān,IR,76.86,17,29.45,55.68,0,5.46,1592436005
337,Naxos,GR,71.01,68,37.11,25.38,0,3.36,1592436019
358,Seydi,TM,73.35,21,39.48,62.91,0,9.24,1592436020
368,Mizdah,LY,76.14,29,31.45,12.98,0,9.62,1592436024
381,São Miguel do Araguaia,BR,73.02,53,-13.28,-50.16,0,5.01,1592436025


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df["Hotel Name"] = ''
targetRadius = 5000
targetSearch = "Hotel"
targetType = "lodging"

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in hotel_df.iterrows():
    params = {
    "location": f'{row["Lat"]}, {row["Long"]}',
    "keyword": targetSearch,
    "type": targetType,
    "radius": targetRadius, 
    "key": g_key
    }
    
    response = requests.get(base_url, params).json()
    
    try: 
        hotelName = response['results'][0]['name']
        hotel_df.loc[index, 'Hotel Name'] = hotelName
        print(hotelName)
        locations = hotel_df[["Lat", "Long"]]
        markers = gmaps.marker_layer(locations)
    except (KeyError, IndexError):
        print("No Hotel Found")
    
hotel_df

Chapadinhense Hotel
Hotel ibis Styles Iu Namibe
HOTEL "AKNUR"
Hotel Kununurra
Desert Gat
Tourism Hotel
Naxos Resort Beach Hotel
No Hotel Found
No Hotel Found
Hotel Veredas do Araguaia
No Hotel Found
Binhe Business Hotel
Aparthotel Mil Cidades
HOTEL E POUSADA A3P
Oiti Hotel


,City,Country,Temp,Humidity,Lat,Long,Cloud,Wind,Date,Hotel Name
50,Corrente,BR,74.66,79,-3.74,-43.34,0,1.66,1592435953,Chapadinhense Hotel
102,Mossamedes,AO,71.44,60,-15.20,12.15,0,6.55,1592435849,Hotel ibis Styles Iu Namibe
137,Beyneu,KZ,77.49,19,45.32,55.20,0,9.66,1592435883,"HOTEL ""AKNUR"""
212,Kununurra,AU,71.60,45,-15.77,128.73,0,8.05,1592435997,Hotel Kununurra
230,Kiryat Gat,IL,75.99,45,31.61,34.76,0,3.71,1592435999,Desert Gat
271,Sīrjān,IR,76.86,17,29.45,55.68,0,5.46,1592436005,Tourism Hotel
337,Naxos,GR,71.01,68,37.11,25.38,0,3.36,1592436019,Naxos Resort Beach Hotel
358,Seydi,TM,73.35,21,39.48,62.91,0,9.24,1592436020,
368,Mizdah,LY,76.14,29,31.45,12.98,0,9.62,1592436024,
381,São Miguel do Araguaia,BR,73.02,53,-13.28,-50.16,0,5.01,1592436025,Hotel Veredas do Araguaia


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Long"]]



In [22]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.symbol_layer(
    locations, fill_color="red",
    stroke_color="blue",
    info_box_content=hotel_info
)
fig.add_layer(markers)
fig.add_layer(hotel_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))